In [3]:
# load .env file
from dotenv import load_dotenv
load_dotenv(override=True)

True

In [4]:
import nest_asyncio
nest_asyncio.apply()

In [5]:
from llama_index.core import SimpleDirectoryReader

# load documents
documents = SimpleDirectoryReader(input_files=["The Era of Experience Paper.pdf"]).load_data()

In [7]:
from llama_index.core.node_parser import SentenceSplitter

splitter = SentenceSplitter(chunk_size=1024)
nodes = splitter.get_nodes_from_documents(documents)

In [9]:
from llama_index.core import Settings
from llama_index.llms.openai import OpenAI
from llama_index.embeddings.openai import OpenAIEmbedding

Settings.llm = OpenAI(model="gpt-4o-mini")
Settings.embed_model = OpenAIEmbedding(model="text-embedding-3-small")

In [10]:
from llama_index.core import SummaryIndex, VectorStoreIndex

summary_index = SummaryIndex(nodes)
vector_index = VectorStoreIndex(nodes)

In [11]:
summary_query_engine = summary_index.as_query_engine(
    response_mode="tree_summarize",
    use_async=True,
)
vector_query_engine = vector_index.as_query_engine()

In [12]:
from llama_index.core.tools import QueryEngineTool


summary_tool = QueryEngineTool.from_defaults(
    query_engine=summary_query_engine,
    description=(
        "Useful for summarization questions related to MetaGPT"
    ),
)

vector_tool = QueryEngineTool.from_defaults(
    query_engine=vector_query_engine,
    description=(
        "Useful for retrieving specific context from the MetaGPT paper."
    ),
)

In [13]:
from llama_index.core.query_engine.router_query_engine import RouterQueryEngine
from llama_index.core.selectors import LLMSingleSelector


query_engine = RouterQueryEngine(
    selector=LLMSingleSelector.from_defaults(),
    query_engine_tools=[
        summary_tool,
        vector_tool,
    ],
    verbose=True
)

In [14]:
response = query_engine.query("What is the summary of the document?")
print(str(response))

Selecting query engine 0: The question asks for a summary of the document, which aligns with the purpose of choice 1, as it is useful for summarization questions..
The document discusses the transition to a new era in artificial intelligence characterized by agents that learn predominantly from their own experiences rather than relying solely on human-generated data. This "Era of Experience" aims to overcome the limitations of current AI systems, which have reached a plateau in performance due to the finite nature of human data. 

Key features of this new era include:

1. **Continuous Learning**: Agents will learn from ongoing streams of experience, allowing them to adapt over time and pursue long-term goals, unlike current systems that operate in short, isolated interactions.

2. **Autonomous Interaction**: Agents will engage with their environments through rich actions and observations, moving beyond human-centric communication to more autonomous behaviors.

3. **Grounded Rewards**: 

In [15]:
print(len(response.source_nodes))

14


In [17]:
response = query_engine.query(
    "What are potential and risks mentioned in the document?",
)
print(str(response))

Selecting query engine 1: The question asks for specific information about potential and risks mentioned in the document, which aligns with retrieving specific context from the MetaGPT paper..
The document outlines several potential benefits and risks associated with the era of experience in AI. 

On the potential side, experiential learning is expected to unlock unprecedented capabilities, such as personalized assistants that adapt to individual needs over time and the acceleration of scientific discovery. AI agents could autonomously design and conduct experiments, rapidly exploring new frontiers in fields like materials science and medicine, leading to the development of novel materials, drugs, and technologies.

Conversely, the risks include job displacement due to increased productivity and the potential misuse of AI. There are heightened concerns regarding agents that can autonomously interact with the world over extended periods, which may reduce human oversight and necessitate 

In [18]:
response.source_nodes

[NodeWithScore(node=TextNode(id_='ceb781fd-efad-4642-b943-aee0380ad7d1', embedding=None, metadata={'page_label': '8', 'file_name': 'The Era of Experience Paper.pdf', 'file_path': 'The Era of Experience Paper.pdf', 'file_type': 'application/pdf', 'file_size': 226196, 'creation_date': '2025-04-21', 'last_modified_date': '2025-04-21'}, excluded_embed_metadata_keys=['file_name', 'file_type', 'file_size', 'creation_date', 'last_modified_date', 'last_accessed_date'], excluded_llm_metadata_keys=['file_name', 'file_type', 'file_size', 'creation_date', 'last_modified_date', 'last_accessed_date'], relationships={<NodeRelationship.SOURCE: '1'>: RelatedNodeInfo(node_id='7523dd9c-986d-4993-93a3-5f73dff80797', node_type='4', metadata={'page_label': '8', 'file_name': 'The Era of Experience Paper.pdf', 'file_path': 'The Era of Experience Paper.pdf', 'file_type': 'application/pdf', 'file_size': 226196, 'creation_date': '2025-04-21', 'last_modified_date': '2025-04-21'}, hash='07b9cbc4593cce78a8ef8a5f17c